In [5]:
import sys
sys.path.append("../agrilearn/submodules/commons/")
from agrilearn.commons.s3 import s3_utils
from agrilearn.crop_classification import evaluate_utils, yaml_utils, processing
import geopandas as gpd
import os
import pandas as pd

In [6]:
#config = yaml_utils.load_config("/agrilearn_app/output/config/experiment_12.yaml")

In [7]:
MAIN_PATH = "/agrilearn_app/datasets/CORN/2021_2022/"
GEOPACKAGE_FILE_PATH = os.path.join(MAIN_PATH, "geopackage/processed/CORN_9467.gpkg")
NEW_EOPATCH_PATH = os.path.join(MAIN_PATH, "eopatch/processed/")
REPORT_PATH = os.path.join(MAIN_PATH, "reports/")
OLD_EOPATCH_PATH = "/agrilearn_app/datasets/CORN/eopatch/processed/"

LABEL_EOPATCH_LOCATION = "eopath_location" 

In [8]:
df = gpd.read_file(GEOPACKAGE_FILE_PATH)

In [9]:
df.shape

(9467, 73)

In [10]:
df.head()

,monitoring_class,period,fonte,state,area,micro,eopath_location,start_season,end_season,peak_start,...,obs_emergence,obs_senescence,obs_harvest,created_by,area_id,last_date_crop_rnn,crop_distance_score_sits,crop_confidence_maha_sits,set,geometry
0,CORN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,RO,None,NaN,start_2021-09-01_end_2022-06-01_monitoring_cla...,2021-09-01,2022-06-01,2021-11-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-63.72032 -10.18348, -63.72032..."
1,CORN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,RO,None,NaN,start_2021-09-01_end_2022-06-01_monitoring_cla...,2021-09-01,2022-06-01,2021-11-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-61.15743 -11.93242, -61.15755..."
2,CORN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,RO,None,NaN,start_2021-09-01_end_2022-06-01_monitoring_cla...,2021-09-01,2022-06-01,2021-11-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-61.07564 -12.36307, -61.07521..."
3,CORN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,PA,None,NaN,start_2021-10-01_end_2022-07-01_monitoring_cla...,2021-10-01,2022-07-01,2021-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-49.52225 -6.21956, -49.52147 ..."
4,CORN,2021/2022,Agrosatélite - Grãos Brasil 2021/2022,PA,None,NaN,start_2021-10-01_end_2022-07-01_monitoring_cla...,2021-10-01,2022-07-01,2021-12-01,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-48.19879 -5.6021, -48.19862 -..."


In [11]:
df_results = processing.check_load_eopatch_and_change_eopatch_dir(df,
                                        eopatch_path=OLD_EOPATCH_PATH,
                                        new_eopatch_path=NEW_EOPATCH_PATH,
                                        label_monitoring_class='monitoring_class',
                                        label_eopatch_path='eopath_location')

  0%|          | 0/9467 [00:00<?, ?it/s]

## Check eopatch

In [12]:
df_processed, df_overview_processed = processing.check_processed_eopatches_features_based_geopackage(df, 
                                                                                         eopatch_path=NEW_EOPATCH_PATH,
                                                                                         label_eopatch_path=LABEL_EOPATCH_LOCATION)

  0%|          | 0/9467 [00:00<?, ?it/s]

In [14]:
df_processed[df_processed['eopatch_processed_exists']==True]

,eopatch_location_id,eopatch_processed_exists,len_data,len_bands,len_time_interval,len_timestamp,error_processed
0,start_2021-09-01_end_2022-06-01_monitoring_cla...,True,2,10,2,29,None
1,start_2021-09-01_end_2022-06-01_monitoring_cla...,True,2,10,2,23,None
2,start_2021-09-01_end_2022-06-01_monitoring_cla...,True,2,10,2,28,None
3,start_2021-10-01_end_2022-07-01_monitoring_cla...,True,2,10,2,22,None
4,start_2021-10-01_end_2022-07-01_monitoring_cla...,True,2,10,2,26,None
...,...,...,...,...,...,...,...
9462,start_2021-09-01_end_2022-07-01_monitoring_cla...,True,2,10,2,29,None
9463,start_2021-09-01_end_2022-07-01_monitoring_cla...,True,2,10,2,42,None
9464,start_2021-09-01_end_2022-07-01_monitoring_cla...,True,2,10,2,41,None
9465,start_2021-09-01_end_2022-07-01_monitoring_cla...,True,2,10,2,25,None


## Merge Data

In [15]:
df['eopath_location'].loc[0]

'start_2021-09-01_end_2022-06-01_monitoring_class_CORN_epsg4326_minxymaxxy_-63dot725036837380124_-10dot18913150801262_-63dot72027784274821_-10dot171379764064795_503_1'

In [16]:
df_processed['eopatch_location_id'].loc[0]

'start_2021-09-01_end_2022-06-01_monitoring_class_CORN_epsg4326_minxymaxxy_-63dot725036837380124_-10dot18913150801262_-63dot72027784274821_-10dot171379764064795_503_1'

In [17]:
# Supondo que df e df_processed já estejam definidos
df_full = pd.merge(df, 
                   df_processed,
                   left_on='eopath_location',
                   right_on='eopatch_location_id',
                   how='left')

In [19]:
df_full.shape

(28913, 80)

In [ ]:
duplicacoes = df_full[df_full.duplicated(subset=['eopath_location'], keep=False)]

In [ ]:
df_full['eopath_location'].nunique()

## Save data

In [ ]:
result_string= '_'.join([f"{cls}_{count}" for cls, count in df_full['monitoring_class'].value_counts().items()])
result_string

In [ ]:
os.makedirs(REPORT_PATH, exist_ok=True)
df_full.to_csv(f"{REPORT_PATH}/overview_{result_string}.csv")

In [ ]:
df_full.to_file(GEOPACKAGE_FILE_PATH, 
                driver='GPKG', 
                engine='fiona')

In [ ]:
# GEOPACKAGE_PATH = "/agrilearn_app/datasets/baselines/testes_de_concordancia/geopackage/SOYBEAN_216_CORN_127.gpkg"
# EOPATCH_PATH = "/agrilearn_app/datasets/baselines/teste_pre_safra_2024_2025/eopatch/processed/"
# NEW_EOPATCH_PATH = "/agrilearn_app/datasets/baselines/testes_de_concordancia/eopatch/processed/"

In [ ]:
# df = gpd.read_file(GEOPACKAGE_PATH)

In [ ]:
# df

In [ ]:
# df_results_v2 = processing.check_load_eopatch_and_change_eopatch_dir(df,
#                                         eopatch_path=EOPATCH_PATH,
#                                         new_eopatch_path=NEW_EOPATCH_PATH,
#                                         label_monitoring_class='monitoring_class',
#                                         label_eopatch_path='eopath_location')